In [1]:
# 导入比特流文件
from pynq import Overlay
overlay = Overlay('./bitstream/Transmit_img.bit')

In [2]:
# ip核
imgprocess = overlay.ImgProcess_0
dma = overlay.axi_dma

In [3]:
# 导入需要的库
import cv2 as cv
import matplotlib.pyplot as plt
import time
import numpy as np
from pynq import Xlnk

In [4]:
# 核心处理模块
# 四叉树分割估算大气光值
def AirlightEstimate(img):
    start = time.time()
    im = cv.cvtColor(I,cv.COLOR_BGR2GRAY)
    imlist = [[],[]]
    for times in range(5):
        h,w = im.shape[0],im.shape[1]
        imlist[0].append(im[0:int(w/2),0:int(h/2)])
        imlist[0].append(im[int(w/2):w,0:int(h/2)])
        imlist[0].append(im[0:int(h/2),int(h/2):h])
        imlist[0].append(im[int(w/2):w,int(h/2):h])
        imlist[1].append(img[0:int(w/2),0:int(h/2)])
        imlist[1].append(img[int(w/2):w,0:int(h/2)])
        imlist[1].append(img[0:int(h/2),int(h/2):h])
        imlist[1].append(img[int(w/2):w,int(h/2):h])
        scorelist = [np.mean(i) for i in imlist[0]]
        max_index = np.argmax(scorelist)
        im,img = imlist[0][max_index],imlist[1][max_index]
        imlist = [[],[]]
    img_single_channel_list = cv.split(img)
    for channel in img_single_channel_list:
        A.append(np.mean(channel))

In [5]:
# 前置处理
path = './images/1.jpeg'
A= []
I = cv.imread(path)
start = time.time()
tmp = cv.resize(I,(120,65))
b,g,r = cv.split(tmp)
height,width = I.shape[0],I.shape[1]
in_array = np.vstack((b,g,r))

In [6]:
# 分配内存
xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(65*3,120), 
                           dtype=np.uint8)
out_buffer = xlnk.cma_array(shape=(65,120), 
                           dtype=np.double)

In [7]:
# 使用dma将数据传到IP核
np.copyto(in_buffer,in_array)
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer) 
imgprocess.write(0x00,0x81) # start 
out_array = np.array(out_buffer)
end = time.time()

In [8]:
d_ref = cv.resize(out_array,(width,height))

In [9]:
AirlightEstimate(I)
I_single_channel_list = cv.split(I)
result = [(I_single_channel_list[i] - A[i]) / d_ref + A[i] for i in range(3)]

In [10]:
dst = cv.merge((result[2],result[1],result[0]))
dst /= 255
dst = np.abs(dst)
dst = np.clip(dst,0,1)
# plt.imshow(dst)
# plt.show()

In [11]:
# 释放内存
in_buffer.close()
out_buffer.close()
xlnk.xlnk_reset()